# Welcome to the WRF-Python and VAPOR Workshop!
September 26-27, 2018

# Jupyter Notebook Quick Reference

**To execute a cell in jupyter notebook**:

1. Click on the desired cell.
2. Press **CTRL + RETURN** to execute the cell or press **SHIFT + RETURN** to execute the cell and advance to the next cell.
3. Alternatively, you can use the Cell dropdown menu

**If you accidentally double click on a Markdown Cell (cells with no code, only pretty text)**.

You will usually see text, and most likely blue '#' characters and letters
1. Simply execute the cell using the instructions above

**If you experience a problem and want to restart your notebook**:

1. Use the Kernel dropdown menu at the top.
2. Execute Kernel -> Restart & Clear Output.
3. Be sure the run **Example 1.1: Verifying your Jupyter Environment** before running any 
   other cells.


**Shutting down the notebook**

1. On your web browser, select the Home tab.
2. Click the check box next to boise_workshop_2018.ipynb.
3. Click the Shutdown button that will become available after step 2.
4. Now go to the terminal window where you typed in "jupyter notebook".
5. With the terminal window active, press **CTRL + C**.


# 2.0 Introduction to jupyter, numpy, xarray

## Example 2.1 Verifying your Jupyter Environment

Looking at your own WRF data is much more fun than my examples!  If you have your own data files, during the Open Lab section, do the following:

1. Place the WRF output files in to a single directory.  Pick two or three files to limit memory.
2. In the cell below, modify the **WRF_DIRECTORY** and **WRF_FILES** variables to point to your data.
3. Execute the cell and verify that "All Tests Passed!" is printed.
4. If not, raise your hand and one of the lab assistants will help.

**IMPORTANT: If for some reason your workbook crashes, you need to run this cell again before running the later examples**.

In [ ]:
from __future__ import print_function

# This jupyter notebook command inserts matplotlib graphics in 
# to the workbook
%matplotlib inline

# Modify these to point to your own files
WRF_DIRECTORY = "~/wrf_tutorial_data"
WRF_FILES = ["wrfout_d01_2010-06-02_00_00_00",
             "wrfout_d01_2010-06-03_00_00_00",
             "wrfout_d01_2010-06-04_00_00_00"]


# Do not modify the code below this line
#------------------------------------------------------
# Turn off annoying warnings
import warnings
warnings.filterwarnings('ignore')

# Make sure the environment is good
import numpy
import cartopy
import matplotlib
from netCDF4 import Dataset
from xarray import DataArray
from wrf import (getvar, interplevel, vertcross, 
                 vinterp, ALL_TIMES)
import os

_WRF_FILES = [os.path.abspath(os.path.expanduser(
    os.path.join(WRF_DIRECTORY, f))) for f in WRF_FILES]

# Check that the WRF files exist
for f in _WRF_FILES:
    if not os.path.exists(f):
        raise ValueError("{} does not exist. "
            "Check for typos or incorrect directory.".format(f))

# Create functions so that the WRF files only need
# to be specified using the WRF_FILES global above
def single_wrf_file():
    global _WRF_FILES
    return _WRF_FILES[0]

def multiple_wrf_files():
    global _WRF_FILES
    return _WRF_FILES

print("All tests passed!")
    


# Numpy Way-Too-Quick Reference

See https://docs.scipy.org/doc/numpy/user/quickstart.html for a much more complete introduction.

## Creating an Array

The zeros function creates an array with all 0's.  

Begin by specifying the desired shape as a tuple and then specify the data type (Note: "float32" is the default if not specified).

``` python
my_array = numpy.zeros((3,3,3), "float32")
```

## Accessing Elements

Use the "[ ]" syntax with the desired dimension indexes separated by commas.

``` python

# Getting an element
first_element = my_array[0,0,0]
last_element = my_array[-1,-1,-1]
mid_element = my_array[1,1,1]

# Setting an element
my_array[1,1,1] = 10.0

```

## Slices

Use the 'start : end' syntax for slicing.  

If *start* is left blank, the slice begins at the start of the array.  

If *end* is blank, the slice ends at the end of the array.

``` python
import numpy

my_array = numpy.zeros((3,3,3), "float32")

first_row = my_array[0,0,:]

first_column = my_array[0,:,0]

first_z = my_array[:,0,0]

subset = my_array[:, :, 1:3]

```

## MaskedArray

Use the *masked_{condition}* functions to create MaskedArray objects.

``` python

import numpy
import numpy.ma

my_array = numpy.zeros((3,3,3), "float32")

# Now all of the array elements are masked values
my_masked = numpy.ma.masked_equal(my_array, 0)

```


## Example 2.2: Numpy Basics

This example demonstrates the basics of creating a numpy array, getting and setting an array element, slicing, and masking values.

In [ ]:
import numpy
import numpy.ma

my_array = numpy.zeros((3,3,3), "float32")

print("my_array")
print(my_array)
print("\n")

# Setting an element
my_array[1,1,1] = 10.0

# Getting an element
mid = my_array[1,1,1]

print("Mid element set")
print(my_array)
print("\n")

# Getting a slice
my_slice = my_array[1,:,:]

print("my_slice")
print(my_slice)
print("\n")

# Masking the zeros
my_masked = numpy.ma.masked_equal(my_array, 0)

print("my_masked")
print(my_masked)
print("\n")

# xarray Basic Way-Too-Quick Reference

See http://xarray.pydata.org/ for a much more complete reference on xarray.

## Creating an xarray DataArray


Arguments for creating a DataArray are:

- data [required]: A numpy array.
- coords [optional]: A dictionary of {dimension_name : coordinate_array}.
- dims [optional]: A list of dimension names from left to right.
- name [optional]: A name for the variable.
- attrs[optional]: A dictionary of {attr_name : attr_value}.

``` python

import numpy
import xarray

my_array = numpy.zeros((3,3,3), "float32")

# Making up dimension names and coordinates.
my_name = "my_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
             "south_north": [40., 50., 60.],
             "west_east" : [-120., -110., -100.]
            }

my_attrs = {"info" : "This is my xarray"}

my_xarray = xarray.DataArray(my_array,
                             name=my_name,
                             dims=my_dims, 
                             coords=my_coords, 
                             attrs=my_attrs)

```

## xarray Arrays with Missing Values

- xarray always uses NaN for missing values.
- This can cause problems with compiled math rouines.
- The xarray.DataArray contains a .values property to get the numpy array, but this will not 
  replace the NaN values with the _FillValue values (see fillna and to_masked_array in xarray documentation).
- wrf-python has a *to_np* function that will handle the conversion to numpy for you and fill in the missing values if _FillValue is in the attributes.

## xarray Notes:

- xarray DataArrays support most of the numpy methods and attributes, but not all.
- xarray DataArrays are NOT numpy subclasses.  
- Often you need to extract the numpy array from the DataArray before passing the array to 
  a computational routine.
- However, the routines in wrf-python are xarray-aware and will do this for you.

## Example 2.3: Creating an xarray DataArray

Run the cell below to see what an xarray DataArray looks like.

In [ ]:
import numpy
import xarray

my_array = numpy.zeros((3,3,3), "float32")

# Making up dimension names and coordinates.
my_name = "my_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
             "south_north": [40., 50., 60.],
             "west_east" : [-120., -110., -100.]
            }

my_attrs = {"info" : "This is my xarray"}

my_xarray = xarray.DataArray(my_array,
                             name=my_name,
                             dims=my_dims, 
                             coords=my_coords, 
                             attrs=my_attrs)

print(my_xarray)

## Example 2.4: xarray and Missing Values

In this example, an xarray.DataArray is constructed from a MaskedArray.  

Note how the missing values are all NaN.

At the end, the wrf-python *to_np* routine is used to convert the DataArray back to a MaskedArray.

In [ ]:
import numpy
import numpy.ma
import xarray

from wrf import to_np

# Create a MaskedArray with 10.0 in the center
my_array = numpy.zeros((3,3,3), "float32")

my_array[1,1,1] = 10.0

my_masked = numpy.ma.masked_equal(my_array, 0)

# Making up dimension names and 
# coordinates.
my_name = "my_masked_xarray"

my_dims = ["bottom_top", "south_north", "west_east"]

my_coords = {"bottom_top" : [100., 200., 300.],
             "south_north": [40., 50., 60.],
             "west_east" : [-120., -110., -100.]
            }

my_attrs = {"info" : "This is my masked xarray",
           "_FillValue" : -999.0}

# Create the xarray DataArray
my_xarray = xarray.DataArray(my_masked,
                             name=my_name,
                             dims=my_dims, 
                             coords=my_coords, 
                             attrs=my_attrs)

print("xarray Array with Missing Values")
print(my_xarray)
print("\n")

# Covert back to a MaskedArray
converted = to_np(my_xarray)

print("Converted to a MaskedArray with to_np")
print(converted)

# 3.0 Overview of WRF-ARW Data

## Example 3.1: Running ncdump

In this example, the ncdump command is called from inside of python.  

Examine the output for your data. 

In [ ]:
from subprocess import Popen, PIPE, STDOUT

file_path = single_wrf_file()

# This simply executes 'ncdump -h {wrf_file}' 
# from Python
p = Popen(["ncdump", "-h", "{}".format(file_path)], stdout=PIPE, stderr=STDOUT)
output, _ = p.communicate()

print(output.decode())

## Example 3.2: Using netcdf4-python

In this example, the netcdf4-python package is used to read your WRF NetCDF file.  


In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

print(wrf_file)

## Example 3.3: Variables, Attributes, and Data with netcd4-python

In this example, netcdf4-python is used to get the global attributes, get a variable (P), get the variable attributes (coordinates), and get the variable data.

In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()

# Create the netCDF4.Dataset object
wrf_file = Dataset(file_path)

# Get the global attribute dict
global_attrs = wrf_file.__dict__
print("Global attributes for the file")
print(global_attrs)
print("\n")

# Just get the 'MAP_PROJ' attribute
map_proj = wrf_file.getncattr("MAP_PROJ")
print("The MAP_PROJ attribute:")
print(map_proj)
print("\n")

# Get the perturbation pressure variable
p = wrf_file.variables["P"]
print("The P variable: ")
print(p)
print("\n")

# Get the P attributes
p_attrs = p.__dict__
print("The attribute dict for P")
print(p_attrs)
print("\n")

# Get the 'coordinates' attribute for P
coords = p.getncattr("coordinates")
print("Coordinates for P:")
print(coords)
print("\n")

# Get the P numpy array for all times
p_all_data = p[:]
print("The P numpy array: ")
print(p_all_data)
print("\n")

# Get the P numpy array for time 0
p_t0_data = p[0,:]
print("P array at time 0:")
print(p_t0_data)
print("\n")

# 4.0 WRF-Python Functions

## Example 4.1: Using getvar to Extract a WRF NetCDF Variable

In this example, the *getvar* function is used to read a variable in a WRF NetCDF file.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

print(hgt)

## Example 4.2: Using getvar to compute Sea Level Pressure (SLP)

In this example, *getvar* is used to compute a diagnostic variable.

The full table of avaiable diagnostics is here:  http://wrf-python.readthedocs.io/en/latest/diagnostics.html

Also try changing the units for by specifiying the following values: 'hPa', 'Pa', 'atm', 'mmhg'

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", timeidx=0, units="hPa")

print(slp)

## Example 4.3: Combining Files Using the 'cat' Method

In this example, the variable is computed for all times, across all files, and combined along the Time dimension by using the 'cat' method

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()
wrf_files = [Dataset(f) for f in file_paths]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="cat")

print(slp)

## Example 4.4: Combining Files Using the 'join' Method

In this example, the variable is computed for all times, across all files, and combined by creating a new leftmost index for the file by using the 'join' method.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()
wrf_files = [Dataset(f) for f in file_paths]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="join")

print(slp)

## Example 4.5: Interpolate to 500 hPa Using interplevel

In this example, the 500 hPa geopotential heights are calculated by using the *interplevel* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, interplevel

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

pres = getvar(wrf_file, "pressure", timeidx=0)
ht = getvar(wrf_file, "z", timeidx=0, units="dm")

ht_500 = interplevel(ht, pres, 500.0)

print(ht_500)

## Example 4.6: Interpolate to a Vertical Cross Section with vertcross

In this example, the vertical cross section for wind speed is calculated using the *vertcross* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vertcross, CoordPair

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Making a diagonal cross section line from 
# bottom left to top right.
bottom_left = CoordPair(x=0, y=0)
top_right = CoordPair(x=-1, y=-1)

# Let's get wind speed in kts
wspd_wdir = getvar(wrf_file, "wspd_wdir", 
                   timeidx=0, units="kt")          
wspd = wspd_wdir[0,:]

# Get the height levels
ht = getvar(wrf_file, "z", timeidx=0)

wspd_cross = vertcross(wspd, ht, 
                       start_point=bottom_left, 
                       end_point=top_right, latlon=True)

print(wspd_cross)

## Example 4.7: Interpolate to Theta-e Levels with vinterp

In this example, pressure is interpolated to various theta-e levels by using the *vinterp* function.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vinterp 

file_path = single_wrf_file()
wrf_file = Dataset(file_path) 

pres = getvar(wrf_file, "pressure", timeidx=0)

# Interpolate pressure to theta-e levels                 
interp_levels = [280., 285., 290., 292., 294., 
                 296., 298., 300., 305., 310.]

pres_eth = vinterp(wrf_file, 
                   field=pres, 
                   vert_coord="theta-e", 
                   interp_levels=interp_levels, 
                   extrapolate=True, 
                   field_type="pressure", 
                   log_p=False,
                   timeidx=0)

print (pres_eth)

## Example 4.8: xy_to_ll and ll_to_xy

In this example, several x,y coordinate values are converted to latitude,longitude values.  

These latitude,longitude values are then converted back to x,y values.

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, xy_to_ll, ll_to_xy 

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

lat_lon = xy_to_ll(wrf_file, [20, 30], [50,75])

print("lat,lon values")
print(lat_lon)
print("\n")

x_y = ll_to_xy(wrf_file, lat_lon[0,:], lat_lon[1,:])

print("x,y values")
print(x_y)


# 5.0 Plotting

## Example 5.1: Single Wind Barb Example with pyplot

In this example, a single wind barb is created in the center of the domain (by masking all other values).

This simple example uses only the matplotlib.pyplot API.

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# Draw a single wind barb in the middle using pyplot API
# Note: the axes objects are "hidden" in these functions
fig = pyplot.figure()
pyplot.barbs(u, v)

# Set the x and y ranges so the barb is in the middle
pyplot.xlim(0, 4)
pyplot.ylim(0, 4)

pyplot.show()

## Example 5.2: Single Wind Barb Using the Axes Object

This is the same example as before, but now the Axes objects are used directly, rather than the pure pyplot API.

Often you'll use a mix of pyplot and the object API when working with matplotlib, especially when making panel plots.

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# We'll use pyplot to create the figure and 
# get the axes
fig = pyplot.figure()
ax = pyplot.axes() # <- Remember this line

# Now use the axes directly to create the barbs
ax.barbs(u, v)

# Set the x and y ranges using the axes directly
ax.set_xlim(0, 4)
ax.set_ylim(0, 4)

pyplot.show()

## Example 5.3: Making a Plot of Terrain

In this example, terrain is plotted.  Plotting terrain is a good way of checking that your data is mapped correctly.

Note:  The first time you run this code, you will need to be connected to the internet so that cartopy can download the map background shapefiles.

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
terrain = getvar(wrf_file, "ter", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(terrain)
lats, lons = latlon_coords(terrain)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(9, 10))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states.
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=2.0, edgecolor='white', zorder=2)

# Set the contour levels
levels = np.arange(250., 4000., 500.)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(terrain), levels=levels, 
               colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(terrain), levels=levels,
                transform=crs.PlateCarree(),
                cmap=get_cmap("terrain"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=1.0)

pyplot.show()

## Example 5.4: Full Plot of Precipitable Water (inches)

In this example, precipitable water is plotted for the full domain.

The following examples show how you can crop this figure.

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get precipitable water in kg/m2 [or mm]
pw = getvar(wrf_file, "pw", 0)

# Get the Precipitable Water in inches 
# [Note: 1 kg/m2 = 1 mm = .0393701 in]
pw_in = pw * .0393701

# After a math operation, xarray drops the attributes, so 
# let's add them back and set the units to be inches
pw_in.attrs.update(pw.attrs)
pw_in.attrs["units"] = "in"


# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(pw_in)
lats, lons = latlon_coords(pw_in)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(9, 10))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states
# See the cartopy documentation for more on this
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=2.0, edgecolor='black', zorder=2)

# Set the contour levels so that all plots match
levels = np.arange(0, 1.4, .2)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(pw_in), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(pw_in), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=1.0)

pyplot.show()

## Example 5.5: Cropping by Slicing the Data

In this example, the data is cropped to the lower right quadrant by slicing the data before plotting.

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get precipitable water in kg/m2 [or mm]
pw = getvar(wrf_file, "pw")

# Get the Precipitable Water in inches 
# [Note: 1 kg/m2 = 1 mm = .0393701 in]
pw_in = pw * .0393701

# After a math operation, xarray drops the attributes, so 
# let's add them back and set the units to be inches
pw_in.attrs.update(pw.attrs)
pw_in.attrs["units"] = "in"

# Determine the center of the domain in grid coordinates
pw_shape = pw_in.shape
center_y = int(pw_shape[-2]/2.) - 1
center_x = int(pw_shape[-1]/2.) - 1

# Slice from bottom to middle for y
# Slice from middle to right for x
pw_quad = pw_in[..., 0:center_y+1, center_x:]

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(pw_quad)
lats, lons = latlon_coords(pw_quad)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(9, 10))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=2.0, edgecolor='black', zorder=2)

# Set the contour levels
levels = np.arange(0, 1.4, .2)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(pw_quad), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(pw_quad), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=1.0)

pyplot.show()

## Example 5.6: Cropping by Setting the X,Y Extents

In this example, the plot is cropped to the lower right quardrant by setting x and y axis extents.

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords
from wrf import xy_to_ll, cartopy_xlim, cartopy_ylim
from wrf import CoordPair, GeoBounds

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get precipitable water in kg/m2 [or mm]
pw = getvar(wrf_file, "pw")

# Get the Precipitable Water in inches 
# [Note: 1 kg/m2 = 1 mm = .0393701 in]
pw_in = pw * .0393701

# After a math operation, xarray drops the attributes, so 
# let's add them back and set the units to be inches
pw_in.attrs.update(pw.attrs)
pw_in.attrs["units"] = "in"

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(pw_in)
lats, lons = latlon_coords(pw_in)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(9, 10))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states
# See the cartopy documentation for more on this
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=2.0, edgecolor='black', zorder=2)

# Set the contour levels
levels = np.arange(0, 1.4, .2)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(pw_in), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(pw_in), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=1.0)

# Set up the x, y extents

# Determine the center of the domain in grid coordinates
pw_shape = pw_in.shape
start_y = 0
center_y = int(pw_shape[-2]/2.) - 1
center_x = int(pw_shape[-1]/2.) - 1
end_x = int(pw_shape[-1]) - 1

# Get the lats and lons for the start, center, and end points
# (Normally you would just set these yourself)
center_latlon = xy_to_ll(wrf_file, 
                         [center_x, end_x], 
                         [start_y, center_y])

start_lat = center_latlon[0,0]
end_lat = center_latlon[0,1]
start_lon = center_latlon[1,0]
end_lon = center_latlon[1,1]

# Set the extents
geo_bounds = GeoBounds(CoordPair(lat=start_lat, lon=start_lon),
                       CoordPair(lat=end_lat, lon=end_lon))
geo_axes.set_xlim(cartopy_xlim(pw_in, geobounds=geo_bounds))
geo_axes.set_ylim(cartopy_ylim(pw_in, geobounds=geo_bounds))

pyplot.show()

# 6.0 OpenMP and Performance

The examples below gradually improve upon performance by using OpenMP, adjusting the scheduler, and using a variable cache.

## Example 6.1: Performance Improvement with OpenMP

In this example, all diagnostics are calculated using one thread, and then using the maximum number of threads available.

In [ ]:
from time import time
from netCDF4 import Dataset
from wrf import (getvar, ALL_TIMES, omp_enabled, 
                 omp_get_num_procs, omp_set_num_threads)

if not omp_enabled():
    raise RuntimeError("OpenMP is not available in this build")

# For this demo, only use the first file since there are 24 time steps
file_path = single_wrf_file()

wrf_file = Dataset(file_path)
    
vars = ("avo", "eth", "cape_2d", "cape_3d", "ctt", "dbz", "mdbz",
        "geopt", "helicity", "lat", "lon", "omg", "p", "pressure",
        "pvo", "pw", "rh2", "rh", "slp", "ter", "td2", "td", "tc",
        "theta", "tk", "tv", "twb", "updraft_helicity", "ua", "va",
        "wa", "uvmet10", "uvmet", "z", "cfrac")    

print("Running with a single CPU")

# Use 1 CPU
omp_set_num_threads(1)

start = time()

for var in vars:
    v = getvar(wrf_file, var, 0)
    
end = time()

print("Time taken: {} s".format(end-start))

print ("Running with {} CPUs".format(omp_get_num_procs()))

# Use Max CPUs
omp_set_num_threads(omp_get_num_procs())

start = time()

for var in vars:
    v = getvar(wrf_file, var, 0)
    
end = time()

print("Time taken: {} s".format(end-start))

## OpenMP Scheduling Types

WRF-Python uses *runtime* scheduling, which means that one of the scheduler types defined below must be set at runtime rather than compile time.

- **wrf.OMP_SCHED_STATIC**: Divide the loop in to equal-sized chunks (default chunk_size = loop_count/num_threads) 
  **[wrf-python default]**
- **wrf.OMP_SCHED_DYNAMIC**: Use internal loop queue to give a chunk-sized block of loop iterations to each thread. 
  When thread is finished, it retrieves next block to work on (default chunk_size = 1).
- **wrf.OMP_SCHED_GUIDED**: Similar to OMP_SCHED_DYNAMIC, but the chunk size starts off large and decreases to better 
  handle load imbalance between iterations. (default chunk_size = loop_count/num_threads)
- **wrf.OMP_SCHED_AUTO**: The decision regarding scheduling is delegated to the OpenMP implementation for the compiler
  (magic!)

## Example 6.2: OpenMP Scheduling Types

In this example, the various scheduler types are compared to see if performance is improved.

In [ ]:
from time import time
from netCDF4 import Dataset
from wrf import (getvar, ALL_TIMES, omp_enabled, 
                 omp_get_num_procs, omp_set_num_threads,
                 omp_set_schedule, OMP_SCHED_STATIC,
                 OMP_SCHED_DYNAMIC, 
                 OMP_SCHED_GUIDED, OMP_SCHED_AUTO)

if not omp_enabled():
    raise RuntimeError("OpenMP is not available in this build")

file_path = single_wrf_file()

wrf_file = Dataset(file_path)
    
vars = ("avo", "eth", "cape_2d", "cape_3d", "ctt", "dbz", "mdbz",
        "geopt", "helicity", "lat", "lon", "omg", "p", "pressure",
        "pvo", "pw", "rh2", "rh", "slp", "ter", "td2", "td", "tc",
        "theta", "tk", "tv", "twb", "updraft_helicity", "ua", "va",
        "wa", "uvmet10", "uvmet", "z", "cfrac")    

omp_set_num_threads(omp_get_num_procs())

chunk_size = 0

sched_string_map = {int(OMP_SCHED_STATIC) : "OMP_SCHED_STATIC", 
                    int(OMP_SCHED_DYNAMIC) : "OMP_SCHED_DYNAMIC",  
                    int(OMP_SCHED_GUIDED) : "OMP_SCHED_GUIDED",
                    int(OMP_SCHED_AUTO) : "OMP_SCHED_AUTO"}

for sched in (OMP_SCHED_STATIC, OMP_SCHED_DYNAMIC, 
              OMP_SCHED_GUIDED, OMP_SCHED_AUTO):
    omp_set_schedule(sched, chunk_size)
    sched_string = sched_string_map[int(sched)]
    print("Running with sheduler: {}".format(sched_string))
    
    start = time()
    for var in vars:
        v = getvar(wrf_file, var, 0)
    end = time()
    
    print("Time taken using scheduler {}: {} s".format(sched_string, end-start))

## WRF-Python and Variable Extraction

- Everytime you call wrf.getvar, the variables needed for the calculation are extracted from the NetCDF file.
- The same variables are going to be extracted over and over and over and over and...
- To prevent this, you can use the *cache* argument to getvar.

## The *cache* Argument to getvar

- Normally used internally so that variables containing metadata aren't extracted more than once.
- Can also be used to prevent the repeated extraction of common variables.
- Should be a dictionary of variable name to variable data.
- Good variables to use: **P, PB, PH, PHB, T, QVAPOR, HGT, PSFC, U, V, W**.

## Example 6.3: Using the cache Argument

In this example, a variable cache is used to show how much performance can be improved by removing the variable extraction cost. The scheduler types are compared as in the previous example.

In [ ]:
from time import time
from netCDF4 import Dataset
from wrf import (getvar, extract_vars, ALL_TIMES, omp_enabled, 
                 omp_get_num_procs, omp_set_num_threads,
                 omp_set_schedule, OMP_SCHED_STATIC,
                 OMP_SCHED_DYNAMIC, 
                 OMP_SCHED_GUIDED, OMP_SCHED_AUTO)

if not omp_enabled():
    raise RuntimeError("OpenMP is not available in this build")

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

print("Caching common variables")
cache = extract_vars(wrf_file, 0, 
                     ("P", "PSFC", "PB", "PH", "PHB",
                      "T", "QVAPOR", "HGT", "U", "V",
                      "W"))
print("Finished caching")
print("----------------")
    
vars = ("avo", "eth", "cape_2d", "cape_3d", "ctt", "dbz", "mdbz",
        "geopt", "helicity", "lat", "lon", "omg", "p", "pressure",
        "pvo", "pw", "rh2", "rh", "slp", "ter", "td2", "td", "tc",
        "theta", "tk", "tv", "twb", "updraft_helicity", "ua", "va",
        "wa", "uvmet10", "uvmet", "z", "cfrac")    

omp_set_num_threads(omp_get_num_procs())

chunk_size = 0

sched_string_map = {int(OMP_SCHED_STATIC) : "OMP_SCHED_STATIC", 
                    int(OMP_SCHED_DYNAMIC) : "OMP_SCHED_DYNAMIC",  
                    int(OMP_SCHED_GUIDED) : "OMP_SCHED_GUIDED",
                    int(OMP_SCHED_AUTO) : "OMP_SCHED_AUTO"}

for sched in (OMP_SCHED_STATIC, OMP_SCHED_DYNAMIC, 
              OMP_SCHED_GUIDED, OMP_SCHED_AUTO):
    omp_set_schedule(sched, chunk_size)
    sched_string = sched_string_map[int(sched)]
    print("Running with sheduler: {}".format(sched_string))
    
    start = time()
    for var in vars:
        v = getvar(wrf_file, var, 0, cache=cache)
    end = time()
    
    print("Time taken using scheduler {}: {} s".format(sched_string, end-start))

# 7.0 Advanced Examples

These examples are a lot more complicated than the previous examples, but you might find them more useful in the real world.  

We might not have time to cover these in the tutorial, so hang on to them for future reference.

## Example 7.1: Overlaying Multiple Diagnostics

In this example, we're going to add winds and dewpoint to sea level pressure.  

[Note: not the prettiest picture for the mountains]

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords, cartopy_xlim, cartopy_ylim

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the slp, td2, u, and v variables
slp = getvar(wrf_file, "slp", timeidx=0)
td2 = getvar(wrf_file, "td2", timeidx=0, units="degF")
u_sfc = getvar(wrf_file, "ua", timeidx=0, units="kt")[0,:]
v_sfc = getvar(wrf_file, "va", timeidx=0, units="kt")[0,:]

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(9, 10))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=2.0, edgecolor='black', zorder=2)


# Manually setting the contour levels
slp_levels = np.arange(980.,1030.,6.0)
td2_levels = np.arange(10., 79., 3.)


# Manually setting the td2 RGB colors (normalized to 1)
# These colors originated from the now defunct hoot.metr.ou.edu
# They work well for detecting moisture boundaries (e.g. the dryline)
td2_rgb = np.array([[181,82,0], [181,82,0],
                  [198,107,8], [206,107,8],
                  [231,140,8], [239,156,8],
                  [247,173,24], [255,189,41],
                  [255,212,49], [255,222,66],
                  [255,239,90], [247,255,123],
                  [214,255,132], [181,231,148],
                  [156,222,156], [132,222,132],
                  [112,222,112], [82,222,82],
                  [57,222,57], [33,222,33],
                  [8,206,8], [0,165,0],
                  [0,140,0], [3,105,3]]) / 255.0
    
td2_cmap, td2_norm = from_levels_and_colors(td2_levels, td2_rgb, extend="both")

# Make the pressure contour lines
slp_contours = pyplot.contour(to_np(lons), 
                              to_np(lats), 
                              to_np(slp), 
                              levels=slp_levels, 
                              colors="black",
                              transform=crs.PlateCarree())

# Make filled contours of dewpoint
pyplot.contourf(to_np(lons), 
                to_np(lats), 
                to_np(td2), 
                levels=td2_levels, 
                cmap=td2_cmap, 
                norm=td2_norm,
                extend="both",
                transform=crs.PlateCarree())

# Plot the wind barbs, but only plot ~10 barbs in each direction.
thin = [int(x/10.) for x in lons.shape]
pyplot.barbs(to_np(lons[::thin[0], ::thin[1]]), 
             to_np(lats[::thin[0], ::thin[1]]), 
             to_np(u_sfc[::thin[0], ::thin[1]]), 
             to_np(v_sfc[::thin[0], ::thin[1]]),
             transform=crs.PlateCarree())

# Add contour labels for pressure
pyplot.clabel(slp_contours, fmt="%i")

# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=1.0, extend="both")

# Set the map bounds
pyplot.xlim(cartopy_xlim(slp))
pyplot.ylim(cartopy_ylim(slp))

pyplot.show()

## Example 7.2: 500 hPa RH and Winds with interplevel

In this example, the 500 hPa relative humidity is plotted with wind barbs.

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import (getvar, to_np, get_cartopy, latlon_coords, interplevel,
                 cartopy_xlim, cartopy_ylim)

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Extract the pressure, relative humidity, and wind variables
p = getvar(wrf_file, "pressure")
rh = getvar(wrf_file, "rh")
ua = getvar(wrf_file, "ua", units="kt")
va = getvar(wrf_file, "va", units="kt")

# Interpolate rh, u, and v winds at the level specified below
level = 500.
rh_level = interplevel(rh, p, level)
u_level = interplevel(ua, p, level)
v_level = interplevel(va, p, level)

# Get the lat/lon coordinates
lats, lons = latlon_coords(rh_level)

# Get the map projection information
cart_proj = get_cartopy(rh_level)

# Create the figure
fig = pyplot.figure(figsize=(9,10))
ax = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
ax.add_feature(states, linewidth=2.0, edgecolor='black', zorder=2)

# Add the RH contour lines
levels = np.arange(0, 110, 10)
contours = pyplot.contourf(to_np(lons), 
                           to_np(lats), 
                           to_np(rh_level), 
                           levels=levels, 
                           cmap=get_cmap("PRGn"),
                           transform=crs.PlateCarree())

pyplot.colorbar(contours, ax=ax, shrink=1.0)

# Add the wind barbs, only plotting 10 barbs in each direction
# Also, skip the border barbs
thin = [int(x/10.) for x in lons.shape]
pyplot.barbs(to_np(lons[::thin[0], ::thin[1]]), 
             to_np(lats[::thin[0], ::thin[1]]), 
             to_np(u_level[::thin[0], ::thin[1]]),
             to_np(v_level[::thin[0], ::thin[1]]), 
             length=6,
             transform=crs.PlateCarree())

# Set the map bounds
ax.set_xlim(cartopy_xlim(rh_level))
ax.set_ylim(cartopy_ylim(rh_level))

ax.gridlines()

pyplot.title("{} MB RH (%) and Barbs (kt)".format(int(level)))

pyplot.show()

## Example 7.3: Precipitation Accumulation

In this example, the precipitation accumulation for one day is plotted every 4 hours. 

The WRF files used for this tutorial use precipitation buckets, which is common for long running simulations, but not usually used for short term simulations. At this time, WRF-Python does not provide a 'getvar' routine for computing this, so we have to do it manually.

To compute precipitation, you need to add the precipitation from the cumulus and microphysics schemes to get the 
total precipitation. Precipitation is accumulated from the start of the WRF simulation. In this data set, every time 100 mm is accumulated, an integer is incremented for the accumulation variables. So, to get the total precipitation, you do:

    TotalAccumPrecip = (I_RAINC * bucket_mm) + RAINC + (I_RAINNC * bucket_mm) + RAINNC


In [ ]:
import numpy as np
import itertools
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors, LinearSegmentedColormap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
from xarray import DataArray
from wrf import (getvar, extract_global_attrs, ALL_TIMES, to_np, get_cartopy, latlon_coords, vertcross,
                 cartopy_xlim, cartopy_ylim, interpline)


file_paths = multiple_wrf_files()
wrf_files = [Dataset(file_path) for file_path in file_paths]

# Get the WRF precipitation variables
rainc = getvar(wrf_files, "RAINC", ALL_TIMES)
rainnc = getvar(wrf_files, "RAINNC", ALL_TIMES)
i_rainc = getvar(wrf_files, "I_RAINC", ALL_TIMES)
i_rainnc = getvar(wrf_files, "I_RAINNC", ALL_TIMES)
bucket_mm = extract_global_attrs(wrf_files, ("BUCKET_MM",))["BUCKET_MM"]

# This is "RAINC + RAINNC" using precip buckets
total_accum = (i_rainc * bucket_mm + rainc) + (i_rainnc * bucket_mm + rainnc)

# Get the precipitation accumulations in 4 hour periods
# This numpy notation is the same as:
#     four_hour_accums[0,:] = total_accum[4, :] - total_accum[0,:]
#     four_hour_accums[1,:] = total_accum[8, :] - total_accum[0,:]
#     four_hour_accums[2,:] = total_accum[12, :] - total_accum[0,:]
#     ...
four_hour_accums = total_accum[4:25:4,:] - total_accum[0,:]

# Convert the precipitation from mm to inches
four_hour_accums = four_hour_accums * 0.0393701

# Let's set the metadata for this variable
four_hour_accums.name = "PRECIP_4HR"
four_hour_accums.attrs.update(rainc.attrs)
four_hour_accums.attrs["description"] = "precip accumulation 4 hour intervals"
four_hour_accums.attrs["units"] = "in"
# Let's make a new secondary coordinate for the time interval labels
four_hour_accums.coords["accum_hrs"] = ("Time", [4, 8, 12, 16, 20, 24])

# Get the lat/lon points
lats, lons = latlon_coords(four_hour_accums)

# Get the cartopy projection object
cart_proj = get_cartopy(four_hour_accums)

# Create the figure
fig = pyplot.figure(figsize=(16,20))

# Download and create the states, land, and oceans using cartopy features
states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none',
                             name='admin_1_states_provinces_shp')
land = NaturalEarthFeature(category='physical', name='land', scale='50m',
                           facecolor=COLORS['land'])

# Making precipitation levels using several groupings of evenly spaced levels
# (Taken from www.twisterdata.com).
precip_levels = np.append(np.arange(.01, .05, .02), np.arange(.05, .30, .05))
precip_levels = np.append(precip_levels, np.arange(.30, 1.0, .1))
precip_levels = np.append(precip_levels, np.arange(1.0, 2.0, .25))
precip_levels = np.append(precip_levels, np.arange(2.0, 5.0, .5))

# Here is how you can truncate an existing colormap in to a subset of it.
orig_cmap = get_cmap("gist_ncar")
cmap = LinearSegmentedColormap.from_list("gist_ncar_trunc", 
                                         orig_cmap(np.linspace(.12, .9, 256)))

# Make the 6 subplots
for plotid in range(6):
    
    # The figure that will have 6 subplots (3 rows, 2 columns)
    # Note: subplot indexes start at 1 instead of 0
    ax = fig.add_subplot(3,2,plotid+1, projection=cart_proj)

    accum = four_hour_accums[plotid,:]
    accum_hr = int(accum["accum_hrs"])
    
    # Make the precip accum contours
    precip_contours = ax.contourf(to_np(lons), 
                                  to_np(lats), 
                                  to_np(accum), 
                                  levels=precip_levels,  
                                  cmap=cmap,
                                  zorder=2,
                                  transform=crs.PlateCarree())


    # Draw the oceans, land, and states
    ax.add_feature(states, linewidth=2.0, edgecolor="white", zorder=3)
    ax.add_feature(land, zorder=1)
    
    cb_precip = fig.colorbar(precip_contours, ax=ax)
    
    ax.set_xlim(cartopy_xlim(accum))
    ax.set_ylim(cartopy_ylim(accum))

    # Add a title
    ax.set_title("Precip Accum (in) After {} Hours".format(accum_hr), {"fontsize" : 12})

pyplot.tight_layout()
pyplot.show()

## Example 7.4: Cross Section Panel Plot

In this example, a panel plot is created with one plot showing maximum refleictivity with the horizontal cross section line, and the second plot is a vertical cross section for radar reflectivity with the terrain heights added to show the mountains.

First, in the cell below, define your cross section in latitude,longitude coordinates.

In [ ]:
from wrf import CoordPair

cross_start = CoordPair(lat=43.5, lon=-116.5)
cross_end = CoordPair(lat=43.5, lon=-114)

Note: If you modified the cell above, the original values were:

``` python
cross_start = CoordPair(lat=43.5, lon=-116.5)
cross_end = CoordPair(lat=43.5, lon=-114.0)
```

Now run the cell below to generate a cross section plot of reflectivity.

(Also remember to run the cell below again if you make changes to the cross section start and end points)

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
from wrf import (getvar, to_np, get_cartopy, latlon_coords, vertcross,
                 cartopy_xlim, cartopy_ylim, interpline)

try:
    cross_start
    cross_end
except NameError:
    raise RuntimeError("you didn't run the previous cell")

file_path = multiple_wrf_files()
wrf_file = [Dataset(x) for x in file_path]

# Get the WRF variables
ht = getvar(wrf_file, "z", timeidx=-1)
ter = getvar(wrf_file, "ter", timeidx=-1)
dbz = getvar(wrf_file, "dbz", timeidx=-1)
max_dbz = getvar(wrf_file, "mdbz", timeidx=-1)
Z = 10**(dbz/10.) # Use linear Z for interpolation

# Compute the vertical cross-section interpolation.  Also, include the lat/lon
# points along the cross-section in the metadata by setting latlon to True.
z_cross = vertcross(Z, ht, wrfin=wrf_file, 
                    start_point=cross_start, 
                    end_point=cross_end,
                    latlon=True, meta=True)

# Convert back to dBz after interpolation
dbz_cross = 10.0 * np.log10(z_cross)

# Add back the attributes that xarray dropped from the operations above
dbz_cross.attrs.update(z_cross.attrs)
dbz_cross.attrs["description"] = "radar reflectivity cross section"
dbz_cross.attrs["units"] = "dBZ"

# To remove the slight gap between the dbz and terrain due to contouring, the new vertical 
# grid spacing, and grid staggering, let's fill in the lower grid cells with the first non-missing 
# value for each column.

# Make a copy of the z cross data. Let's use regular numpy arrays for this.
dbz_cross_filled = np.ma.copy(to_np(dbz_cross))

# For each cross section column, find the first index with non-missing values and copy
# these to the missing elements below.
for i in range(dbz_cross_filled.shape[-1]):
    column_vals = dbz_cross_filled[:,i]
    # Let's find all values that aren't filled. The nonzero function checks for nonzero 
    # values, but 0s are ok for reflectivity, so let's just get all values greater 
    # than some nonsensical negative and use nonzero() on the boolean array. 
    first_idx = int(np.transpose((column_vals > -200.).nonzero())[0])
    dbz_cross_filled[0:first_idx, i] = dbz_cross_filled[first_idx, i]
            
# Get the terrain heights along the cross section line
ter_line = interpline(ter, wrfin=wrf_file, start_point=cross_start, end_point=cross_end)

# Get the lat/lon points
lats, lons = latlon_coords(dbz)

# Get the cartopy projection object
cart_proj = get_cartopy(dbz)

# Create a figure that will have 2 subplots (1 row, 2 columns)
fig = pyplot.figure(figsize=(14,7))
ax_dbz = fig.add_subplot(1,2,1,projection=cart_proj)
ax_cross = fig.add_subplot(1,2,2)

# Download and create the states and land
states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none',
                             name='admin_1_states_provinces_shp')
land = NaturalEarthFeature(category='physical', name='land', scale='50m',
                           facecolor=COLORS['land'])

dbz_levels = np.arange(5., 80., 5.)

# This is the NWS color table.
dbz_rgb = np.array([[4,233,231],
                    [1,159,244], [3,0,244],
                    [2,253,2], [1,197,1],
                    [0,142,0], [253,248,2],
                    [229,188,0], [253,149,0],
                    [253,0,0], [212,0,0],
                    [188,0,0],[248,0,253],
                    [152,84,198],[253,253,253]], np.float32) / 255.0
    
dbz_map, dbz_norm = from_levels_and_colors(dbz_levels, dbz_rgb, extend="max")

# Make the dbz contours on the map
dbz_contours = ax_dbz.contourf(to_np(lons), 
                    to_np(lats), 
                    to_np(max_dbz), 
                    levels=dbz_levels,
                    cmap=dbz_map, 
                    norm=dbz_norm, 
                    extend="max",
                    zorder=3, 
                    transform=crs.PlateCarree())

# Draw the cross section line
ax_dbz.plot([cross_start.lon, cross_end.lon], 
            [cross_start.lat, cross_end.lat],
            color="black",
            linewidth=3.0,
            marker="o",  
            zorder=5,
            transform=crs.PlateCarree())

# Draw the oceans, land, and states
ax_dbz.add_feature(states, linewidth=2.0, edgecolor="black", zorder=3)
ax_dbz.add_feature(land)

# Make the cross section plot for dbz
dbz_levels = np.arange(5.,75.,5.)
xs = np.arange(0, dbz_cross.shape[-1], 1)
ys = to_np(dbz_cross.coords["vertical"])
dbz_contours = ax_cross.contourf(xs, 
                                 ys, 
                                 to_np(dbz_cross_filled), 
                                 levels=dbz_levels,
                                 cmap=dbz_map, 
                                 norm=dbz_norm, 
                                 extend="max")
cb_dbz = fig.colorbar(dbz_contours, ax=ax_cross)
cb_dbz.ax.tick_params(labelsize=8)

# Fill in the mountain area
ht_fill = ax_cross.fill_between(xs, 0, to_np(ter_line), facecolor="saddlebrown")

# Set the x-ticks to use latitude and longitude labels
coord_pairs = to_np(dbz_cross.coords["xy_loc"])
x_ticks = np.arange(coord_pairs.shape[0])
x_labels = [pair.latlon_str() for pair in to_np(coord_pairs)]

# Set the desired number of x ticks below
num_ticks = 5
thin = int((len(x_ticks) / num_ticks) + .5)
ax_cross.set_xticks(x_ticks[::thin])
ax_cross.set_xticklabels(x_labels[::thin], rotation=45, fontsize=8)

# Set the x-axis and  y-axis labels
ax_cross.set_xlabel("Latitude, Longitude", fontsize=12)
ax_cross.set_ylabel("Height (m)", fontsize=12)

# Add a title
ax_dbz.set_title("Maximum Reflectivity (dBZ)", {"fontsize" : 14})
ax_cross.set_title("Cross-Section of Reflectivity (dBZ)", {"fontsize" : 14})

pyplot.tight_layout()
pyplot.show()

## Example 7.5: Animations

In this example, an animation of radar reflectivity is generated.

To make animations, the ffmpeg package can be used to make movies, but for this example we're going to make a Javascript animation.

To create an animation, you have to define a function that generates the frames (in this example it is named *animate*).

You pass this animation function to the FuncAnimation object.

You must also supply some kind of generator that passes the arguments to the animate function, or you can just supply an integer and the FuncAnimation object will use the range() function.  In this case, we're just generating integer values which represents the Time index.

To render the animation in a jupyter notebook, we make use of the HTML object from Ipython.display and the *to_jshtml* method for the FuncAnimation object to make the Javascript frame animator. If you have ffmpeg installed, you can use the *to_html5_video* to make a movie.

Below the animation will also be a static image of the first frame for radar reflectivity. I haven't figured out how to prevent this, so don't worry about it for now.

In [ ]:
import numpy as np
from matplotlib import pyplot, rc
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
from wrf import (getvar, to_np, get_cartopy, latlon_coords, vertcross,
                 cartopy_xlim, cartopy_ylim, ALL_TIMES, extract_vars,
                 omp_set_num_threads, omp_get_num_procs)

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

print("Calculating dbz...")

# Get DBZ for all times
cache = extract_vars(wrf_file, ALL_TIMES, ("T", "P", "PB", "QVAPOR", 
                                           "QRAIN", "QSNOW", "QGRAUP"))

omp_set_num_threads(omp_get_num_procs())

dbz_all = getvar(wrf_file, "mdbz", timeidx=ALL_TIMES, cache=cache)

# Get the cartopy projection object
cart_proj = get_cartopy(dbz_all)

fig = pyplot.figure(figsize=(8,10))
ax = pyplot.axes(projection=cart_proj)

# Download and create the states and land
states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none',
                             name='admin_1_states_provinces_shp')
land = NaturalEarthFeature(category='physical', name='land', scale='50m',
                           facecolor=COLORS['land'])

dbz_levels = np.arange(5, 80, 5)
num_frames = dbz_all.shape[0]

# This is the NWS radar color table
dbz_rgb = np.array([[4,233,231],
                    [1,159,244], [3, 0, 244],
                    [2,253,2], [1,197,1],
                    [0,142,0], [253,248,2],
                    [229,188,0], [253,149,0],
                    [253,0,0], [212,0,0],
                    [188,0,0],[248,0,253],
                    [152,84,198],[253,253,253]], np.float32) / 255.0
    
dbz_map, dbz_norm = from_levels_and_colors(dbz_levels, dbz_rgb, extend="max")


# This init function is used to set up the colorbar, otherwise it gets 
# repeatedly drawn.
def init():
    
    dbz = dbz_all[0,:]
    
    lats, lons = latlon_coords(dbz)
    
    dbz_contours = ax.contourf(to_np(lons), 
                               to_np(lats), 
                               to_np(dbz), 
                               levels=dbz_levels,
                               cmap=dbz_map, 
                               norm=dbz_norm, 
                               extend="max",
                               zorder=3,
                               transform=crs.PlateCarree())
    
    cb = fig.colorbar(dbz_contours, ax=ax, shrink=.9)
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(dbz))
    ax.set_ylim(cartopy_ylim(dbz))
    
    return ax.clear()
    

print ("Creating animation. This may take a few minutes...")

# This function is called for each frame of the animation, where
# i is the frame index. Here is where the animation frames need 
# to be created.
def animate(i):
    if (i%9 == 0 and i > 0):
        print("Completed frame {}...".format((10*i)//9))
        
    ax.clear()
    
    dbz = dbz_all[i,:]
    
    # Get the lat/lon coordinates
    lats, lons = latlon_coords(dbz)
    
    ax.add_feature(land)
    ax.add_feature(states, linewidth=2.0, edgecolor="black", zorder=3)
    
    dbz_contours = ax.contourf(to_np(lons), 
                               to_np(lats), 
                               to_np(dbz),
                               levels=dbz_levels,
                               cmap=dbz_map, 
                               norm=dbz_norm,
                               extend="max",
                               zorder=3,
                               transform=crs.PlateCarree()) 
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(dbz))
    ax.set_ylim(cartopy_ylim(dbz))
     
    return ax

# Create the animation by supplying a figure, the animation object, the number of frames,
# the init functions, and an interval in milliseconds that is the delay between frames.
ani = FuncAnimation(fig, animate, num_frames, init_func=init, interval=500)

# To work with jupyter notebook, you need to use the HTML generated
# by the HTML function from the IPython.display package.
# If you change 'to_jshtml' to be 'to_html5_video', you will get an HTML5 video instead.

#HTML(ani.to_html5_video())
HTML(ani.to_jshtml())



# Example 7.6: Reducing WRF File Size

The WRF files used for this tutorial were originally 10 GB each. To reduce the files sizes, I removed all of the variables that aren't of interest to WRF-Python, cropped the horizontal domain to a smaller region around Idaho, and removed the vertical levels above 400 mb. 

Until frontend xarray support is added in the future, the only way you can reduce a large WRF data set to a smaller domain is to use a custom interable that crops the domain out and write the files to a temporary location. 

Below is the iterable class I used for this purpose via WRF-Python. 

In [ ]:
import tempfile
import glob
import shutil
import os
import numpy as np
from netCDF4 import Dataset
from wrf import getvar, ll_to_xy, CoordPair, GeoBounds, to_np

_VARS_TO_KEEP = ("Times", "XLAT", "XLONG", "XLAT_U", "XLAT_V", "XLONG_U", 
                "XLONG_V", "U", "V", "W", "PH", "PHB", "T", "P", "PB", "Q2", 
                "T2", "PSFC", "U10", "V10", "XTIME", "QVAPOR", "QCLOUD", 
                "QGRAUP", "QRAIN", "QSNOW", "QICE", "MAPFAC_M", "MAPFAC_U",
                "MAPFAC_V", "F", "HGT", "RAINC", "RAINSH", "RAINNC", "I_RAINC", "I_RAINNC")

class FileReduce(object):
    def __init__(self, filenames, geobounds, tempdir=None, vars_to_keep=None, 
                 max_pres=None, compress=False, delete=True, reuse=False):
        """An iterable object for cutting out geographic domains.
        
        Args:
        
            filenames (sequence): A sequence of file paths to the WRF files
            
            geobounds (GeoBounds): A GeoBounds object defining the region of interest
            
            tempdir (str): The location to store the temporary cropped data files. If None, tempfile.mkdtemp is used.
            
            vars_to_keep (sequence): A sequence of variables names to keep from the original file. None for all vars.
            
            max_press (float): The maximum pressure height level to keep. None for all levels.
            
            compress(bool): Set to True to enable zlib compression of variables in the output.
            
            delete (bool): Set to True to delete the temporary directory when FileReduce is garbage collected.
            
            reuse (bool): Set to True when you want to resuse the files that were previously converted. *tempdir* 
                must be set to a specific directory that contains the converted files and *delete* must be False.
                
        
        """
        self._filenames = filenames
        self._i = 0
        self._geobounds = geobounds
        self._delete = delete
        self._vars_to_keep = vars_to_keep
        self._max_pres = max_pres
        self._compress = compress
        self._cache = set()
        self._own_data = True
        self._reuse = reuse
        
        if tempdir is not None:
            if not os.path.exists(tempdir):
                os.makedirs(tempdir)
            self._tempdir = tempdir
            if self._reuse:
                self._cache = set((os.path.join(self._tempdir, name) 
                                   for name in os.listdir(self._tempdir)))
        else:
            self._tempdir = tempfile.mkdtemp()

        self._prev = None
        self._set_extents()
    
    def _set_extents(self):
        fname = list(self._filenames)[0]
        with Dataset(fname) as ncfile:
            lons = [self._geobounds.bottom_left.lon, self._geobounds.top_right.lon]
            lats = [self._geobounds.bottom_left.lat, self._geobounds.top_right.lat]
            orig_west_east = len(ncfile.dimensions["west_east"])
            orig_south_north = len(ncfile.dimensions["south_north"])
            orig_bottom_top = len(ncfile.dimensions["bottom_top"])
            
            # Note: Not handling the moving nest here
            # Extra points included around the boundaries to ensure domain is fully included
            x_y = ll_to_xy(ncfile, lats, lons, meta=False)
            self._start_x = 0 if x_y[0,0] == 0 else x_y[0,0] - 1
            self._end_x = orig_west_east - 1 if x_y[0,1] >= orig_west_east - 1 else x_y[0,1] + 1
            self._start_y = 0 if x_y[1,0] == 0 else x_y[1,0] - 1
            self._end_y = orig_south_north - 1 if x_y[1,1] >= orig_south_north - 1 else x_y[1,1] + 1
            
            self._west_east = self._end_x - self._start_x + 1
            self._west_east_stag = self._west_east + 1
            self._south_north = self._end_y - self._start_y + 1
            self._south_north_stag = self._south_north + 1
            
            # Crop the vertical to the specified pressure
            if self._max_pres is not None:
                pres = getvar(ncfile, "pressure")
                # Find the lowest terrain height
                ter = to_np(getvar(ncfile, "ter"))
                min_ter = float(np.amin(ter)) + 1
                ter_less = ter <= min_ter
                ter_less = np.broadcast_to(ter_less, pres.shape)
                # For the lowest terrain height, find the lowest vertical index to meet 
                # the desired pressure level. The lowest terrain height will result in the 
                # largest vertical spread to find the pressure level.
                x = np.transpose(((pres.values <= self._max_pres) & ter_less).nonzero())
                self._end_bot_top = np.amin(x, axis=0)[0] 
                if (self._end_bot_top >= orig_bottom_top):
                    self._end_bot_top = orig_bottom_top - 1
            else:
                self._end_bot_top = orig_bottom_top - 1
                
            self._bottom_top = self._end_bot_top + 1
            self._bottom_top_stag = self._bottom_top + 1
            
        
    def __iter__(self):
        return self
    
    def __copy__(self):
        cp = type(self).__new__(self.__class__)
        cp.__dict__.update(self.__dict__)
        cp._own_data = False
        cp._delete = False
        
        return cp
    
    def __del__(self):
        if self._delete:
            shutil.rmtree(self._tempdir)
    
    def reduce(self, fname):
        outfilename = os.path.join(self._tempdir, os.path.basename(fname))
        
        # WRF-Python can iterate over sequences several times during a 'getvar', so a cache is used to 
        if outfilename in self._cache:
            return Dataset(outfilename)
        
        # New dimension sizes
        dim_d = {"west_east" : self._west_east,
                 "west_east_stag" : self._west_east_stag,
                 "south_north" : self._south_north,
                 "south_north_stag" : self._south_north_stag,
                 "bottom_top" : self._bottom_top,
                 "bottom_top_stag" : self._bottom_top_stag
                }
        
        # Data slice sizes for the 2D dimensions
        slice_d = {"west_east" : slice(self._start_x, self._end_x + 1),
                   "west_east_stag" : slice(self._start_x, self._end_x + 2),
                   "south_north" : slice(self._start_y, self._end_y + 1),
                   "south_north_stag" : slice(self._start_y, self._end_y + 2),
                   "bottom_top" : slice(None, self._end_bot_top + 1),
                   "bottom_top_stag" : slice(None, self._end_bot_top + 2)
                  }
        
        with Dataset(fname) as infile, Dataset(outfilename, mode="w") as outfile:
            
            # Copy the global attributes
            outfile.setncatts(infile.__dict__)

            # Copy Dimensions, limiting south_north and west_east to desired domain
            for name, dimension in infile.dimensions.items():
                dimsize = dim_d.get(name, len(dimension))
                outfile.createDimension(name, dimsize)

            # Copy Variables  
            for name, variable in infile.variables.iteritems():
                if self._vars_to_keep is not None:
                    if name not in self._vars_to_keep:
                        continue
                
                print (name)
                new_slices = tuple((slice_d.get(dimname, slice(None)) for dimname in variable.dimensions))

                outvar = outfile.createVariable(name, variable.datatype, variable.dimensions, zlib=self._compress)

                outvar[:] = variable[new_slices]

                outvar.setncatts(variable.__dict__)
                
        
        result = Dataset(outfilename)
            
        self._cache.add(outfilename)
            
        return result
            
    
    def next(self):
        if self._i >= len(self._filenames):
            if self._prev is not None:
                self._prev.close()
            raise StopIteration
        else:
            fname = self._filenames[self._i]
            reduced_file = self.reduce(fname)
            if self._prev is not None:
                self._prev.close()
            self._prev = reduced_file
            
            self._i += 1
            
            return reduced_file
    
    # Python 3
    def __next__(self):
        return self.next()


To use this with getvar, you would do:

``` python

# How to use with getvar

VARS_TO_KEEP = ("Times", "XLAT", "XLONG", "XLAT_U", "XLAT_V", "XLONG_U", 
                "XLONG_V", "U", "V", "W", "PH", "PHB", "T", "P", "PB", "Q2", 
                "T2", "PSFC", "U10", "V10", "XTIME", "QVAPOR", "QCLOUD", 
                "QGRAUP", "QRAIN", "QSNOW", "QICE", "MAPFAC_M", "MAPFAC_U",
                "MAPFAC_V", "F", "HGT", "RAINC", "RAINSH", "RAINNC", 
                "I_RAINC", "I_RAINNC")

# Set lower left and upper right to your desired domain
# Idaho bounding box: ["41.9880561828613","49.000846862793","-117.243034362793","-111.043563842773"]
ll = CoordPair(lat=41.8, lon=-117.26)
ur = CoordPair(lat=49.1, lon=-110.5)

bounds = GeoBounds(ll, ur)

reduced_files = FileReduce(glob.glob("/path/to/wrfout_d01_*"),
                           bounds, 
                           vars_to_keep=VARS_TO_KEEP, 
                           max_pres=400,
                           tempdir="/path/to/reduced", 
                           delete=False, 
                           reuse=True)

pres = getvar(reduced_files, "pressure")

```

Note that by setting 'reuse=True', the second time your run this, it will use the already cropped files, so you only pay the cropping penalty once.